# Quick overview of gymnasium
[Gymnasium website](https://gymnasium.farama.org/)

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
import gymnasium as gym
import pygame

## Simple example from the website

In [ ]:
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(10):
    action = env.action_space.sample()  # this is where you would insert your policy
    observation, reward, terminated, truncated, info = env.step(action)
    print(observation, reward, terminated, truncated, info)
    if terminated or truncated:
        observation, info = env.reset()
env.close()